In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
#Starting from the last data point in the database. 
prev_year = dt.date(2017, 8, 23)- dt.timedelta(days=365)
# Calculate the date one year from the last date in data set.

# Perform a query to retrieve the data and precipitation scores
results = []
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year).all()
#print(results)
# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results, columns=['date','precipitation'])
df.set_index(df['date'], inplace=True)
#print(df.to_string(index=False))
# Sort the dataframe by date
df = df.sort_index()
#print(df.to_string(index=False))
# Use Pandas Plotting with Matplotlib to plot the data
#rot for xticks
df.plot(rot=90)

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

In [ ]:
# How many stations are available in this dataset?
session.query(func.count(Station.station)).all()

In [ ]:
# What are the most active stations?
# List the stations and the counts in descending order.
session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
filter(Measurement.station == 'USC00519281').all()

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
results = session.query(Measurement.tobs).\
filter(Measurement.station == 'USC00519281').\
filter(Measurement.date >= prev_year).all()
df = pd.DataFrame(results, columns=['tobs'])
#print(results)
#print(df)
plt.tight_layout()
df.plot.hist(bins=20)

In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates


# Challenge

In [54]:
results = []
#month of interest June
m=6
#querry both precipitation and temperature
results = session.query(Measurement.date, Measurement.prcp,Measurement.tobs).all()
#print(results)
# Save the query results as a Pandas DataFrame 
df = pd.DataFrame(results, columns=['date','precipitation','temperature'])
df['date'] = pd.to_datetime(df['date'])
#print(df.head())
df1 = df.loc[(df['date'].dt.month==m)]
if m == 6:
    print('JUNE')
else:
    print('DECEMBER')
df1.describe()

JUNE


,precipitation,temperature
count,1574.000000,1700.000000
mean,0.136360,74.944118
std,0.335731,3.257417
min,0.000000,64.000000
25%,0.000000,73.000000
50%,0.020000,75.000000
75%,0.120000,77.000000
max,4.430000,85.000000


In [55]:
results = []
m=12
results = session.query(Measurement.date, Measurement.prcp,Measurement.tobs).all()
#print(results)
# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results, columns=['date','precipitation','temperature'])
df['date'] = pd.to_datetime(df['date'])
#print(df.head())
df1 = df.loc[(df['date'].dt.month==m)]
if m == 6:
    print('JUNE')
else:
    print('DECEMBER')
df1.describe()

DECEMBER


,precipitation,temperature
count,1405.000000,1517.000000
mean,0.216819,71.041529
std,0.541399,3.745920
min,0.000000,56.000000
25%,0.000000,69.000000
50%,0.030000,71.000000
75%,0.150000,74.000000
max,6.420000,83.000000


**Observation**

1. From the tables above december is wetter and cooler 
in the island of **OHAHU** compared to the month of June
2. Standard deviation for temperature is a lot smaller , 
indicating the temperature flucutuations are smaller. 
Seems the range is about 11 deg F for each month
3. Precipitation is almost 60% higher in decmber compared
to June gthroughout the 7 years.
4. Variation in precipaitation for each month is a lot, 
especially for both months june and december  have somedays 
that are very wet but from the quartiles, the number of days 
is probably small. i.e the number of days the business could be slow.

**Recommendation for further Analysis**

1. Look at data by station to see if there are discrpency due to some stations 
reporting erroneous data, filter them out if necessary

2. Analyze by station to see if there are locations in OHAHU
that is better to set up shop

3. Analyze data for the school holiday months, e,g spring break in march,
summer months June,July and August, december break December and in particular 
the last two weeks in december/1 week in January.